In [1]:
import json
import pandas as pd
import numpy as np
import codecs
import re
from collections import defaultdict

In [2]:
df = pd.read_json(codecs.open('../data/trulia_1br_nyc.json','r','utf-8'))
df.head()

,address,apt_features,days_on_trulia,elementary_school_count,high_school_count,middle_school_count,pets_allowed,price,sqft,url,zip_code
0,"990 Avenue Of The Americas, New York, NY","[Deposit: $500, Concierge, Balcony, Garage, At...",63,42,12,22,"Cats, small dogs, large dogs","$2,585 - $5,600","461-1,192",https://www.trulia.com/c/ny/new-york/the-vogue...,10018
1,"377 East 33rd Street, New York, NY","[Fitness Center, On-site Maintenance, Washing ...",125,42,15,22,"Cats, small dogs, large dogs","$4,244 - $7,424","497-1,155",https://www.trulia.com/c/ny/new-york/the-lanth...,10016
10,"66 West 38th Street #116F, Manhattan, NY","[Concierge, Elevator, Fitness Center, Dishwash...",2,42,12,22,Small dogs\n ...,"$3,795",None,https://www.trulia.com/p/ny/manhattan/66-w-38t...,10018
100,"435 West 45th Street #1B, Manhattan, NY","[Guarantor Ok, Guarantors, Live In Super]",2,42,11,22,Cats\n allowed,"$1,950",500,https://www.trulia.com/p/ny/manhattan/435-w-45...,10036
1000,"56 West 11th Street #5H, Manhattan, NY","[Elevator, Dishwasher, Fios Available, Guarant...",14,42,16,19,No pets allowed,"$4,550",768,https://www.trulia.com/p/ny/manhattan/56-w-11t...,10011


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7717 entries, 0 to 999
Data columns (total 11 columns):
address                    7717 non-null object
apt_features               7717 non-null object
days_on_trulia             7717 non-null object
elementary_school_count    7717 non-null int64
high_school_count          7717 non-null int64
middle_school_count        7717 non-null int64
pets_allowed               7717 non-null object
price                      7717 non-null object
sqft                       2768 non-null object
url                        7717 non-null object
zip_code                   7717 non-null object
dtypes: int64(3), object(8)
memory usage: 723.5+ KB


In [4]:
# get a list of links used to scrape trulia pages collected on 04/20/2018

with open("../trulia_rentals/trulia_1br_nyc.txt", "rt") as f:
        start_urls = [url.strip() for url in f.readlines()]

## Data Cleaning

In [5]:
# check for duplicate urls in search page links

print('Number of urls in the list:', len(start_urls))
print('Number of scraped urls:    ',len(df))

Number of urls in the list: 8333
Number of scraped urls:     7717


In [6]:
# QA ZIP codes and remove non 5-digit zips

df['zip_code'].unique()

array(['10018', '10016', '10036', '10011', '10006', '10012', '10025',
       '10021', '10075', '10009', '10014', '10002', '10128', '10019',
       '10034', '10038', '10027', '10028', '10017', '10029', '10010',
       'York', '10033', '10026', '10023', '10024', '10003', '10032',
       '10065', '10022', '10044', '10040', '10001', '10069', '10031',
       '10013', '10035', '10030', '10007', '10282', '10005', 'Manhattan',
       '10039', '10037', '10004', '10280', '11205', '11221', '11211',
       '11215', '11222', '11217', '11232', '11204', '11238', '11214',
       '11223', '11220', '11225', '11226', '11229', '11228', '11201',
       '11203', '11206', '11249', '11237', '11224', '11209', '11210',
       '11218', '11235', 'Brooklyn', '11208', '11207', '11233', '11219',
       '11231', '11213', '11236', '11216', '11234', '11212', '11230',
       '11432', '11691', '11103', '11367', '11368', '11435', '11361',
       '11102', '11358', '11418', 'Queens', 'Jamaica', '11421', '11415',
       '113

In [7]:
# QA ZIP codes and remove non 5-digit zips

index_list = []
for idx, row in df['zip_code'].iteritems():
    try:    
        if str(row) == re.match(r"(^\d{5}$)", str(row)).group(1):
            pass
    except:
        index_list.append(idx)
        print('Row to fix: {}'.format(idx))

Row to fix: 1022
Row to fix: 1025
Row to fix: 1093
Row to fix: 1111
Row to fix: 1173
Row to fix: 1278
Row to fix: 137
Row to fix: 1385
Row to fix: 1387
Row to fix: 1390
Row to fix: 1397
Row to fix: 1398
Row to fix: 1415
Row to fix: 1416
Row to fix: 146
Row to fix: 1478
Row to fix: 1504
Row to fix: 1537
Row to fix: 1573
Row to fix: 1574
Row to fix: 1584
Row to fix: 168
Row to fix: 179
Row to fix: 1874
Row to fix: 1876
Row to fix: 198
Row to fix: 2067
Row to fix: 2092
Row to fix: 2104
Row to fix: 212
Row to fix: 2319
Row to fix: 2410
Row to fix: 2492
Row to fix: 2511
Row to fix: 2545
Row to fix: 2641
Row to fix: 3131
Row to fix: 3253
Row to fix: 3256
Row to fix: 3262
Row to fix: 3291
Row to fix: 33
Row to fix: 3315
Row to fix: 3333
Row to fix: 3341
Row to fix: 3348
Row to fix: 3358
Row to fix: 3405
Row to fix: 3416
Row to fix: 3424
Row to fix: 3448
Row to fix: 3462
Row to fix: 3485
Row to fix: 3489
Row to fix: 355
Row to fix: 3563
Row to fix: 3564
Row to fix: 3593
Row to fix: 3615
Row to

In [8]:
# check what position the zip code is at in the url
df.loc[index_list[0],'url']

'https://www.trulia.com/rental/4015536909-Multi-Family-Home-New-York-NY-10019'

In [9]:
def fix_zip(url):
    new_zip = url.split('-')[-1]
    return(new_zip)

In [10]:
for idx in index_list:
    df.loc[idx,'zip_code'] = fix_zip(df.loc[idx,'url'])

In [11]:
df['zip_code'] = df['zip_code'].astype(int)
df['zip_code'].unique()

array([10018, 10016, 10036, 10011, 10006, 10012, 10025, 10021, 10075,
       10009, 10014, 10002, 10128, 10019, 10034, 10038, 10027, 10028,
       10017, 10029, 10010, 10033, 10026, 10023, 10024, 10003, 10032,
       10065, 10022, 10044, 10040, 10001, 10069, 10031, 10013, 10035,
       10030, 10007, 10282, 10005, 10280, 10039, 10037, 10004, 11205,
       11221, 11211, 11215, 11222, 11217, 11232, 11204, 11238, 11214,
       11223, 11220, 11225, 11226, 11229, 11228, 11201, 11203, 11206,
       11249, 11237, 11224, 11209, 11210, 11218, 11235, 11236, 11208,
       11207, 11233, 11219, 11231, 11213, 11234, 11216, 11212, 11230,
       11432, 11691, 11103, 11367, 11368, 11435, 11361, 11102, 11358,
       11418, 11357, 11421, 11415, 11375, 11378, 11354, 11372, 11379,
       11356, 11106, 11355, 11426, 11377, 11101, 11385, 11369, 11363,
       11428, 11423, 11370, 11419, 11364, 11417, 11366, 11365, 11374,
       11694, 11360, 11104, 11422, 11427, 11434, 11416, 11693, 11362,
       11429, 11373,

In [12]:
df['address'].head()

0       990 Avenue Of The Americas, New York, NY
1             377 East 33rd Street, New York, NY
10      66 West 38th Street #116F, Manhattan, NY
100      435 West 45th Street #1B, Manhattan, NY
1000      56 West 11th Street #5H, Manhattan, NY
Name: address, dtype: object

In [13]:
# remove APT # from address
df['address'] = [re.sub('\s*#\S+,',',',address) for address in df['address']]

In [14]:
# remove zip code from the back of the address
df['address'] = [re.match(r'.*(NY)', address).group(0) for address in df.address]

# remove addresses that only contain borough and state
df = df[df['address'].str[0].str.match('^\d')]

In [15]:
# QA days_on_trulia
df['days_on_trulia'] = [int(str(d_num).replace(' day on Trulia','')) for d_num in df['days_on_trulia']]

In [16]:
# check data for duplicate records

print('Number of duplicate records:', 
      len(df[df.duplicated(
          subset=['address', 'pets_allowed', 'price', 'sqft', 'zip_code'],
          keep= 'first') == True]))


df.drop_duplicates(subset=['address', 'pets_allowed', 'price', 'sqft', 'zip_code'], keep='first', inplace = True)

Number of duplicate records: 212


In [17]:
# drop listings for non multi-bdr apts (they include price-range) and with no price
df = df[df.price.str.contains("-") == False].copy()

df = df[df['price']!=""]

In [18]:
# convert price string to int
df['price'] = [re.sub(r"[$,]","",price) for price in df.price]
df['price'] = df['price'].astype(int)

### apt_features QA

In [19]:
def lower_case(los):
    lower_los = list(map(lambda x: x.lower(), los))
    return(lower_los)

In [20]:
# process apt_features

def count_features(df):
    features = []

    for idx, row in df.iterrows():
        features.extend(row['apt_features'])
        
    feat_count = defaultdict(int)

    for i in features:
        feat_count[i]+=1
    
    feat_count = pd.DataFrame(list(feat_count.values()), index=feat_count.keys(), columns=['count'])
    feat_count = feat_count.sort_values(by='count', ascending=False)
    
    return(feat_count)

In [21]:
# function to map feature 

def feature_maping(feature_list, string):
    feature = 0
    for x in feature_list:
        if x.find(string)>-1:
            feature = 1

    return(feature)

In [22]:
# create a function to remove feature by name

def remove_feature(feature_list, string):
    new_features = []
    for x in feature_list:
        if x.find(string)>-1:
            pass
        else:
            new_features.append(x)
    return(new_features)

In [23]:
# create a function to map evacuation zone from features

def map_evacuation_zone(feature_list):
    evac_zone = None
    for x in feature_list:
        if x.find("evacuation")>-1:
            evac_zone = x.split(' ')[-1]

    return(evac_zone)

In [24]:
def map_pets(string):   
    if string.lower().find('no')>-1:
        pets = 0
    else:
        pets = 1
    return(pets)

In [25]:
# map features 
df['apt_features'] = df.apt_features.apply(lower_case)


#evacuation zones
df['evac'] = df.apt_features.apply(map_evacuation_zone)

# map features
feat_to_map = ['guarantor', 'storage', 'garage', 'deck']

for feat in feat_to_map:
    df[feat] = [feature_maping(feature_list, feat) for feature_list in df['apt_features']]
    
df['pets_allowed'] = [map_pets(string) for string in df.pets_allowed]

# remoevd mapped features and remove garbage
garbage = ['deposit', 'shared','in unit','storage','none', 'evacuation', 'inc', 'refrigerator']
feat_to_remove = feat_to_map + garbage

for feat in feat_to_remove:
    df['apt_features'] = [remove_feature(feature_list, feat) for feature_list in df.apt_features]

In [26]:
# get top features

top_features = list(count_features(df)[:30].index)
top_features

['elevator',
 'dishwasher',
 'pets',
 'fios available',
 'live in super',
 'fitness center',
 'washing machine',
 'dryer',
 'concierge',
 'balcony',
 'air conditioning',
 'flooring: hardwood',
 'recreation facilities',
 'garden',
 'pied a terre',
 'childrens playroom',
 'terrace',
 'courtyard',
 'pool',
 'furnished',
 'fireplace',
 'sublets',
 'valet parking',
 'off-street parking',
 'cable tv ready',
 'board approval required',
 'parents',
 'co purchase',
 'leed registered',
 'microwave oven']

In [27]:
# create a binary column for each top-feature

for feat in top_features:
    df[feat] = [feature_maping(feature_list, feat) for feature_list in df.apt_features]
    
# drop feature column
df.drop(columns=['apt_features'], axis = 1, inplace = True)

In [28]:
df.describe()

,days_on_trulia,elementary_school_count,high_school_count,middle_school_count,pets_allowed,price,zip_code,guarantor,storage,garage,...,fireplace,sublets,valet parking,off-street parking,cable tv ready,board approval required,parents,co purchase,leed registered,microwave oven
count,6297.000000,6297.000000,6297.000000,6297.000000,6297.000000,6297.000000,6297.000000,6297.000000,6297.000000,6297.000000,...,6297.000000,6297.000000,6297.000000,6297.000000,6297.000000,6297.000000,6297.000000,6297.000000,6297.000000,6297.000000
mean,27.054947,35.786247,8.689376,19.931396,0.292203,2716.250119,10616.685406,0.251072,0.287756,0.164205,...,0.040972,0.035572,0.029855,0.027315,0.023344,0.020962,0.020168,0.019215,0.018421,0.015722
std,40.320801,11.210164,4.421891,6.785284,0.454811,1529.801929,600.208697,0.433664,0.452753,0.370491,...,0.198241,0.185236,0.170202,0.163011,0.151007,0.143270,0.140587,0.137293,0.134480,0.124407
min,-1.000000,15.000000,2.000000,6.000000,0.000000,390.000000,10001.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,26.000000,5.000000,15.000000,0.000000,1800.000000,10022.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15.000000,35.000000,8.000000,19.000000,0.000000,2450.000000,10458.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,34.000000,42.000000,12.000000,23.000000,1.000000,3290.000000,11222.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,889.000000,64.000000,28.000000,41.000000,1.000000,71850.000000,11694.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [29]:
# replace whitespaces with underscore in column names
df_cols = list(df.columns)

for c in df:
    new_name = c.replace(" ", "_")
    df.rename(columns={c: new_name}, inplace=True)

In [30]:
df.columns

Index(['address', 'days_on_trulia', 'elementary_school_count',
       'high_school_count', 'middle_school_count', 'pets_allowed', 'price',
       'sqft', 'url', 'zip_code', 'evac', 'guarantor', 'storage', 'garage',
       'deck', 'elevator', 'dishwasher', 'pets', 'fios_available',
       'live_in_super', 'fitness_center', 'washing_machine', 'dryer',
       'concierge', 'balcony', 'air_conditioning', 'flooring:_hardwood',
       'recreation_facilities', 'garden', 'pied_a_terre', 'childrens_playroom',
       'terrace', 'courtyard', 'pool', 'furnished', 'fireplace', 'sublets',
       'valet_parking', 'off-street_parking', 'cable_tv_ready',
       'board_approval_required', 'parents', 'co_purchase', 'leed_registered',
       'microwave_oven'],
      dtype='object')

## Merge with zip code-level income data

In [31]:
# data source: https://www.incomebyzipcode.com/newyork  - scraped with beatifulsoup

income = pd.read_csv('../data/income_data_by_zip.csv')
income.rename(columns={"Unnamed: 0": "zip_code"}, inplace=True)
income.head()

,zip_code,median_income,pct_high_income
0,10030,33720,4.0
1,11226,45031,2.6
2,10305,74063,6.0
3,10280,146875,37.6
4,11208,37640,1.8


In [32]:
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 3 columns):
zip_code           167 non-null int64
median_income      167 non-null int64
pct_high_income    167 non-null float64
dtypes: float64(1), int64(2)
memory usage: 4.0 KB


In [33]:
# join with trulia data
df = pd.merge(df, income, how='left', on=['zip_code'])

In [34]:
df.head()

,address,days_on_trulia,elementary_school_count,high_school_count,middle_school_count,pets_allowed,price,sqft,url,zip_code,...,valet_parking,off-street_parking,cable_tv_ready,board_approval_required,parents,co_purchase,leed_registered,microwave_oven,median_income,pct_high_income
0,"66 West 38th Street, Manhattan, NY",2,42,12,22,1,3795,None,https://www.trulia.com/p/ny/manhattan/66-w-38t...,10018,...,0,0,0,0,0,0,0,0,105479.0,24.4
1,"435 West 45th Street, Manhattan, NY",2,42,11,22,1,1950,500,https://www.trulia.com/p/ny/manhattan/435-w-45...,10036,...,0,0,0,0,0,0,0,0,83812.0,18.1
2,"56 West 11th Street, Manhattan, NY",14,42,16,19,0,4550,768,https://www.trulia.com/p/ny/manhattan/56-w-11t...,10011,...,0,0,0,0,0,0,0,0,114134.0,28.5
3,"298 Mulberry Street, Manhattan, NY",2,40,13,17,0,5350,None,https://www.trulia.com/p/ny/manhattan/298-mulb...,10012,...,0,0,0,0,0,0,0,0,101815.0,21.6
4,"71 West 107th Street, Manhattan, NY",14,27,6,15,0,2475,None,https://www.trulia.com/p/ny/manhattan/71-w-107...,10025,...,0,0,0,0,0,0,0,0,80025.0,19.2


In [35]:
# check if there is missing income data for any zip codes

df[pd.isnull(df.median_income)]['zip_code'].unique()

array([11249])

In [36]:
# grab income data from another source: https://newyork.hometownlocator.com/zip-codes/data

df.loc[df.zip_code == 11249, 'median_income'] = 50436

## Merge with zip code-level demo data

In [37]:
# source U.S. Census Bureau: https://factfinder.census.gov
# 2012-2016 American Community Survey 5-Year Estimates

demo = pd.read_csv('../data/demo_by_zip.csv')
demo.head()

,zip_code,Total population,Male,Female,Under 5 years,5 to 9 years,10 to 14 years,15 to 19 years,20 to 24 years,25 to 34 years,...,PCT_Two or more races,PCT_One race.1,PCT_White,PCT_Black or African American,PCT_American Indian and Alaska Native,PCT_Asian,PCT_Native Hawaiian and Other Pacific Islander,PCT_Some other race,PCT_Two or more races.1,PCT_Hispanic or Latino (of any race)
0,10013,"28,211","13,988","14,223","1,739","1,323",951,"1,107","2,010","5,919",...,0.051788,0.948212,0.560526,0.048598,0.001276,0.320407,0.000000,0.017405,0.051788,0.071781
1,10030,"30,358","14,530","15,828","2,147","1,420","1,468","1,816","2,636","6,436",...,0.059095,0.940905,0.159068,0.653139,0.004381,0.023190,0.002273,0.098854,0.059095,0.271625
2,11226,"101,787","45,676","56,111","7,107","6,464","6,565","6,237","7,386","18,509",...,0.034189,0.965811,0.125596,0.702919,0.001513,0.032087,0.000108,0.103589,0.034189,0.172802
3,10305,"41,533","19,853","21,680","2,502","2,455","2,384","2,067","2,510","6,128",...,0.025642,0.974358,0.760552,0.049310,0.009318,0.115378,0.000433,0.039366,0.025642,0.166903
4,10280,"9,845","4,921","4,924",815,688,479,208,312,"1,853",...,0.028949,0.971051,0.770340,0.015846,0.000000,0.165566,0.000000,0.019299,0.028949,0.112443


In [38]:
# clean up colum names in demo so it's easier to identify during modeling
# lowercase, append 'demo' to the front, replace spaces

demo_cols = list(demo.columns)

for c in demo_cols[1:]:
    new_name = 'demo_' + c.replace(" ", "_").lower()
    demo.rename(columns={c: new_name}, inplace=True)

In [39]:
demo.columns

Index(['zip_code', 'demo_total_population', 'demo_male', 'demo_female',
       'demo_under_5_years', 'demo_5_to_9_years', 'demo_10_to_14_years',
       'demo_15_to_19_years', 'demo_20_to_24_years', 'demo_25_to_34_years',
       'demo_35_to_44_years', 'demo_45_to_54_years', 'demo_55_to_59_years',
       'demo_60_to_64_years', 'demo_65_to_74_years', 'demo_75_to_84_years',
       'demo_85_years_and_over', 'demo_median_age_(years)',
       'demo_18_years_and_over', 'demo_21_years_and_over',
       'demo_62_years_and_over', 'demo_65_years_and_over',
       'demo_18_years_and_over.1', 'demo_male.1', 'demo_female.1',
       'demo_65_years_and_over.1', 'demo_male.2', 'demo_female.2',
       'demo_one_race', 'demo_two_or_more_races', 'demo_one_race.1',
       'demo_white', 'demo_black_or_african_american',
       'demo_american_indian_and_alaska_native', 'demo_asian',
       'demo_native_hawaiian_and_other_pacific_islander',
       'demo_some_other_race', 'demo_two_or_more_races.1',
       'dem

In [40]:
# join demo with trulia data
df = pd.merge(df, demo, how='left', on=['zip_code'])
df.head()

,address,days_on_trulia,elementary_school_count,high_school_count,middle_school_count,pets_allowed,price,sqft,url,zip_code,...,demo_pct_two_or_more_races,demo_pct_one_race.1,demo_pct_white,demo_pct_black_or_african_american,demo_pct_american_indian_and_alaska_native,demo_pct_asian,demo_pct_native_hawaiian_and_other_pacific_islander,demo_pct_some_other_race,demo_pct_two_or_more_races.1,demo_pct_hispanic_or_latino_(of_any_race)
0,"66 West 38th Street, Manhattan, NY",2,42,12,22,1,3795,None,https://www.trulia.com/p/ny/manhattan/66-w-38t...,10018,...,0.039550,0.960450,0.591078,0.054936,0.000000,0.293474,0.000000,0.020962,0.039550,0.165118
1,"435 West 45th Street, Manhattan, NY",2,42,11,22,1,1950,500,https://www.trulia.com/p/ny/manhattan/435-w-45...,10036,...,0.020125,0.979875,0.642167,0.074845,0.004579,0.192712,0.000000,0.065571,0.020125,0.177435
2,"56 West 11th Street, Manhattan, NY",14,42,16,19,0,4550,768,https://www.trulia.com/p/ny/manhattan/56-w-11t...,10011,...,0.029680,0.970320,0.794004,0.045889,0.001270,0.101004,0.000079,0.028073,0.029680,0.109456
3,"298 Mulberry Street, Manhattan, NY",2,40,13,17,0,5350,None,https://www.trulia.com/p/ny/manhattan/298-mulb...,10012,...,0.038882,0.961118,0.756643,0.026125,0.001549,0.161355,0.000489,0.014958,0.038882,0.058730
4,"71 West 107th Street, Manhattan, NY",14,27,6,15,0,2475,None,https://www.trulia.com/p/ny/manhattan/71-w-107...,10025,...,0.041946,0.958054,0.659955,0.121467,0.008341,0.091821,0.000232,0.076238,0.041946,0.227076


## Merge with Yelp data

In [41]:
# Yelp data: the total number of restaurants within 1-mile Walking from an apt address
# was scraped from Yelp using selenium
yelp = pd.read_csv('../data/yelp_rest_data.csv')
yelp.head()

,address,link,rest_num
0,119 West 69th Street Manhattan NY,https://www.yelp.com/search?find_loc=119 West ...,455
1,250 West 90th Street Manhattan NY,https://www.yelp.com/search?find_loc=250 West ...,389
2,24060 65th Avenue Douglaston NY,https://www.yelp.com/search?find_loc=24060 65t...,24
3,321 61st Street Brooklyn NY,https://www.yelp.com/search?find_loc=321 61st ...,395
4,7525 153rd Street Flushing NY,https://www.yelp.com/search?find_loc=7525 153r...,96


In [42]:
# remove commas from address in trulia data to merge on Yelp data
df['address'] = [address.replace(',', '') for address in df.address]
# drop 'link column from yelp
yelp.drop(labels='link', axis=1, inplace=True)
# merge df and yelp on address
df = pd.merge(df, yelp, how='left', on=['address'])
df.head()

,address,days_on_trulia,elementary_school_count,high_school_count,middle_school_count,pets_allowed,price,sqft,url,zip_code,...,demo_pct_one_race.1,demo_pct_white,demo_pct_black_or_african_american,demo_pct_american_indian_and_alaska_native,demo_pct_asian,demo_pct_native_hawaiian_and_other_pacific_islander,demo_pct_some_other_race,demo_pct_two_or_more_races.1,demo_pct_hispanic_or_latino_(of_any_race),rest_num
0,66 West 38th Street Manhattan NY,2,42,12,22,1,3795,None,https://www.trulia.com/p/ny/manhattan/66-w-38t...,10018,...,0.960450,0.591078,0.054936,0.000000,0.293474,0.000000,0.020962,0.039550,0.165118,2879
1,435 West 45th Street Manhattan NY,2,42,11,22,1,1950,500,https://www.trulia.com/p/ny/manhattan/435-w-45...,10036,...,0.979875,0.642167,0.074845,0.004579,0.192712,0.000000,0.065571,0.020125,0.177435,1783
2,56 West 11th Street Manhattan NY,14,42,16,19,0,4550,768,https://www.trulia.com/p/ny/manhattan/56-w-11t...,10011,...,0.970320,0.794004,0.045889,0.001270,0.101004,0.000079,0.028073,0.029680,0.109456,2131
3,298 Mulberry Street Manhattan NY,2,40,13,17,0,5350,None,https://www.trulia.com/p/ny/manhattan/298-mulb...,10012,...,0.961118,0.756643,0.026125,0.001549,0.161355,0.000489,0.014958,0.038882,0.058730,2328
4,71 West 107th Street Manhattan NY,14,27,6,15,0,2475,None,https://www.trulia.com/p/ny/manhattan/71-w-107...,10025,...,0.958054,0.659955,0.121467,0.008341,0.091821,0.000232,0.076238,0.041946,0.227076,418


In [43]:
# check for missing data at a zip code level
df[pd.isnull(df.demo_total_population)]['zip_code'].unique()

array([11249])

## Merge with Business Patters data

In [44]:
# source U.S. Census Bureau: https://factfinder.census.gov
# ZIP Code Business Statistics: Total for Zip Code (2015 Business Patterns)

bispat = pd.read_csv('../data/business_patterns/zbp15totals.txt')
bispat.head()

,zip,name,empflag,emp_nf,emp,qp1_nf,qp1,ap_nf,ap,est,city,stabbr,cty_name
0,501,"HOLTSVILLE, NY",B,D,0,D,0,D,0,2,HOLTSVILLE,NY,SUFFOLK
1,1001,"AGAWAM, MA",NaN,G,9641,G,87217,G,376465,457,AGAWAM,MA,HAMPDEN
2,1002,"AMHERST, MA",NaN,H,18764,G,108300,G,454968,571,AMHERST,MA,HAMPSHIRE
3,1003,"AMHERST, MA",NaN,G,267,H,2038,H,8716,18,AMHERST,MA,HAMPSHIRE
4,1004,"AMHERST, MA",NaN,H,165,H,1179,H,5693,13,AMHERST,MA,HAMPSHIRE


In [45]:
# keep only zip and number of establishments columns
bispat = bispat[['zip','est']].copy()
bispat.rename(columns={'zip': 'zip_code', 'est':'n_of_establishments'}, inplace=True)

In [46]:
# merge on zip
df = pd.merge(df, bispat, how='left', on=['zip_code'])
df.head()

,address,days_on_trulia,elementary_school_count,high_school_count,middle_school_count,pets_allowed,price,sqft,url,zip_code,...,demo_pct_white,demo_pct_black_or_african_american,demo_pct_american_indian_and_alaska_native,demo_pct_asian,demo_pct_native_hawaiian_and_other_pacific_islander,demo_pct_some_other_race,demo_pct_two_or_more_races.1,demo_pct_hispanic_or_latino_(of_any_race),rest_num,n_of_establishments
0,66 West 38th Street Manhattan NY,2,42,12,22,1,3795,None,https://www.trulia.com/p/ny/manhattan/66-w-38t...,10018,...,0.591078,0.054936,0.000000,0.293474,0.000000,0.020962,0.039550,0.165118,2879,6723
1,435 West 45th Street Manhattan NY,2,42,11,22,1,1950,500,https://www.trulia.com/p/ny/manhattan/435-w-45...,10036,...,0.642167,0.074845,0.004579,0.192712,0.000000,0.065571,0.020125,0.177435,1783,6131
2,56 West 11th Street Manhattan NY,14,42,16,19,0,4550,768,https://www.trulia.com/p/ny/manhattan/56-w-11t...,10011,...,0.794004,0.045889,0.001270,0.101004,0.000079,0.028073,0.029680,0.109456,2131,4002
3,298 Mulberry Street Manhattan NY,2,40,13,17,0,5350,None,https://www.trulia.com/p/ny/manhattan/298-mulb...,10012,...,0.756643,0.026125,0.001549,0.161355,0.000489,0.014958,0.038882,0.058730,2328,2972
4,71 West 107th Street Manhattan NY,14,27,6,15,0,2475,None,https://www.trulia.com/p/ny/manhattan/71-w-107...,10025,...,0.659955,0.121467,0.008341,0.091821,0.000232,0.076238,0.041946,0.227076,418,2027


In [47]:
# check for missing data at a zip code level
df[pd.isnull(df.n_of_establishments)]['zip_code'].unique()

array([], dtype=int64)

In [48]:
df.to_pickle('../data/df_all.pkl')

In [49]:
df = pd.read_pickle('../data/df_all.pkl')

In [50]:
df.head()

,address,days_on_trulia,elementary_school_count,high_school_count,middle_school_count,pets_allowed,price,sqft,url,zip_code,...,demo_pct_white,demo_pct_black_or_african_american,demo_pct_american_indian_and_alaska_native,demo_pct_asian,demo_pct_native_hawaiian_and_other_pacific_islander,demo_pct_some_other_race,demo_pct_two_or_more_races.1,demo_pct_hispanic_or_latino_(of_any_race),rest_num,n_of_establishments
0,66 West 38th Street Manhattan NY,2,42,12,22,1,3795,None,https://www.trulia.com/p/ny/manhattan/66-w-38t...,10018,...,0.591078,0.054936,0.000000,0.293474,0.000000,0.020962,0.039550,0.165118,2879,6723
1,435 West 45th Street Manhattan NY,2,42,11,22,1,1950,500,https://www.trulia.com/p/ny/manhattan/435-w-45...,10036,...,0.642167,0.074845,0.004579,0.192712,0.000000,0.065571,0.020125,0.177435,1783,6131
2,56 West 11th Street Manhattan NY,14,42,16,19,0,4550,768,https://www.trulia.com/p/ny/manhattan/56-w-11t...,10011,...,0.794004,0.045889,0.001270,0.101004,0.000079,0.028073,0.029680,0.109456,2131,4002
3,298 Mulberry Street Manhattan NY,2,40,13,17,0,5350,None,https://www.trulia.com/p/ny/manhattan/298-mulb...,10012,...,0.756643,0.026125,0.001549,0.161355,0.000489,0.014958,0.038882,0.058730,2328,2972
4,71 West 107th Street Manhattan NY,14,27,6,15,0,2475,None,https://www.trulia.com/p/ny/manhattan/71-w-107...,10025,...,0.659955,0.121467,0.008341,0.091821,0.000232,0.076238,0.041946,0.227076,418,2027
